In [ ]:
# Helper function to convert objects to dictionaries
def to_dict(obj):
    """Convert object to dictionary"""
    if isinstance(obj, dict):
        return obj
    if hasattr(obj, 'model_dump'):
        return obj.model_dump()
    if hasattr(obj, 'dict'):
        return obj.dict()
    if hasattr(obj, '__dict__'):
        return {k: v for k, v in obj.__dict__.items() if not k.startswith('_')}
    return {}

# Query the agent
print("=" * 60)
print("Querying Agent")
print("=" * 60)

query = "What collections are in the mcp_demo database?"
print(f"\n👤 Query: {query}\n")

# Create turn with streaming
try:
    turn_stream = client.alpha.agents.turn.create(
        agent_id=agent.agent_id,
        session_id=session.session_id,
        messages=[{"role": "user", "content": query}],
        stream=True
    )
    print("✅ Turn created, streaming response...")
except Exception as e:
    print(f"\n❌ Error creating turn: {e}")
    raise

# Extract turn_id from streaming response
print("\n📋 Extracting turn_id from stream...")
turn_id = None
for chunk in turn_stream:
    if hasattr(chunk, 'event') and chunk.event and hasattr(chunk.event, 'payload'):
        payload = chunk.event.payload
        d = to_dict(payload)
        turn_id = d.get('turn_id')
        if not turn_id and 'step_details' in d:
            step_details = to_dict(d['step_details'])
            turn_id = step_details.get('turn_id')
        if turn_id:
            break

if turn_id:
    print(f"✅ Turn ID: {turn_id}")
else:
    print("⚠️  Could not extract turn_id from stream")


In [ ]:
# Retrieve turn results
import json

if turn_id:
    print("=" * 60)
    print("Retrieving Turn Results")
    print("=" * 60)
    
    print("\n⏳ Waiting for turn to complete...")
    time.sleep(2)  # Give the turn time to complete
    
    try:
        # Retrieve the turn
        response = httpx.get(
            f"{llamastack_url}/v1alpha/agents/{agent.agent_id}/session/{session.session_id}/turn/{turn_id}",
            verify=False,
            timeout=30
        )
        response.raise_for_status()
        
        # Handle null response
        if not response.text or response.text.strip() == 'null':
            print("   ⚠️  Turn not ready yet (response was null)")
            print("   💡 The turn may still be processing. Try waiting a bit longer.")
            data = None
        else:
            data = response.json()
        
        if data is None:
            print("   ⚠️  Could not retrieve turn data")
            print("   💡 This might mean the turn is still processing or the turn_id is incorrect")
        else:
            print("✅ Turn data retrieved successfully!")
            
            # Extract messages and steps
            messages = data.get('messages', [])
            steps = data.get('steps', [])
            
            print(f"\n📊 Turn Summary:")
            print(f"   Messages: {len(messages)}")
            print(f"   Steps: {len(steps)}")
            
            # Debug: Print all steps for full visibility
            if steps:
                print(f"\n{'='*60}")
                print("📦 Full Steps Debug (All Step Objects)")
                print(f"{'='*60}\n")
                
                for i, step in enumerate(steps):
                    step_type = step.get('type', '') or step.get('step_type', '')
                    print(f"{'='*60}")
                    print(f"Step {i}: type='{step_type}'")
                    print(f"{'='*60}")
                    print(json.dumps(step, indent=2, default=str))
                    print()
            else:
                print("\n⚠️  No steps found in turn data")
                print("\n📦 Full Data Object (for debugging):")
                data_str = json.dumps(data, indent=2, default=str)
                if len(data_str) > 2000:
                    print(data_str[:2000] + "...")
                else:
                    print(data_str)
            
    except Exception as e:
        print(f"\n❌ Error retrieving turn: {e}")
        import traceback
        traceback.print_exc()
        data = None
else:
    print("\n⚠️  Cannot retrieve turn results without turn_id")
    data = None


In [ ]:
# Analyze tool usage from steps (tool-specific only)
import json

if data:
    print("=" * 60)
    print("Analyzing Tool Usage")
    print("=" * 60)
    
    steps = data.get('steps', [])
    
    if steps:
        # Filter and show only tool-related steps
        tool_steps = []
        for i, step in enumerate(steps):
            step_type = step.get('type', '') or step.get('step_type', '')
            
            # Check for tool calls in inference steps
            if step_type == 'inference':
                model_response = step.get('model_response', {})
                tool_calls = model_response.get('tool_calls', [])
                if tool_calls:
                    tool_steps.append((i, step, 'tool_call', tool_calls))
            
            # Check for tool execution steps
            if step_type == 'tool_execution':
                tool_execution = step.get('tool_execution', {})
                if tool_execution:
                    tool_steps.append((i, step, 'tool_execution', tool_execution))
            
            # Check for tool result steps
            if step_type == 'tool_result':
                tool_result = step.get('tool_result', {})
                if tool_result:
                    tool_steps.append((i, step, 'tool_result', tool_result))
        
        if tool_steps:
            print(f"\n🔧 Found {len(tool_steps)} tool-related step(s):\n")
            
            for step_idx, step, tool_type, tool_data in tool_steps:
                print(f"{'='*60}")
                print(f"Step {step_idx}: {tool_type}")
                print(f"{'='*60}")
                
                if tool_type == 'tool_call':
                    print("\n📋 Tool Calls from Inference Step:")
                    for tool_call in tool_data:
                        call_id = tool_call.get('call_id', 'unknown')
                        tool_name = tool_call.get('tool_name', 'unknown')
                        arguments = tool_call.get('arguments', '{}')
                        print(f"\n  🔧 Tool: {tool_name}")
                        print(f"     Call ID: {call_id}")
                        try:
                            args_dict = json.loads(arguments) if isinstance(arguments, str) else arguments
                            print(f"     Arguments: {json.dumps(args_dict, indent=6)}")
                        except:
                            print(f"     Arguments: {arguments}")
                
                elif tool_type == 'tool_execution':
                    print("\n⚙️  Tool Execution:")
                    tool_name = tool_data.get('tool_name', 'unknown')
                    call_id = tool_data.get('call_id', 'unknown')
                    print(f"  Tool: {tool_name}")
                    print(f"  Call ID: {call_id}")
                    if 'arguments' in tool_data:
                        args = tool_data.get('arguments', {})
                        print(f"  Arguments: {json.dumps(args, indent=4, default=str)}")
                
                elif tool_type == 'tool_result':
                    print("\n📤 Tool Result:")
                    tool_name = tool_data.get('name', tool_data.get('tool_name', 'unknown'))
                    result_content = tool_data.get('content', '')
                    print(f"  Tool: {tool_name}")
                    if result_content:
                        result_str = str(result_content)
                        if len(result_str) > 500:
                            print(f"  Result: {result_str[:500]}...")
                            print(f"  (Full result: {len(result_str)} characters)")
                        else:
                            print(f"  Result: {result_str}")
                
                print()
        else:
            print("\n⚠️  No tool-related steps found")
            print("   💡 The agent may not have used any tools, or tool information is in a different format")
            print("\n   Available step types:")
            for i, step in enumerate(steps):
                step_type = step.get('type', '') or step.get('step_type', '')
                print(f"     Step {i}: {step_type}")
    else:
        print("\n⚠️  No steps found in turn data")
else:
    print("\n⚠️  No turn data available")
    print("   💡 Make sure Part 5 completed successfully")


In [ ]:
# Extract the agent's response
if data:
    print("=" * 60)
    print("Extracting Agent Response")
    print("=" * 60)
    
    messages = data.get('messages', [])
    steps = data.get('steps', [])
    response_text = None
    
    # Try messages first
    print("\n🔍 Checking messages...")
    for msg in reversed(messages):
        if msg.get('role') == 'assistant':
            content = msg.get('content', '')
            if isinstance(content, str) and content.strip():
                response_text = content
                print("   ✅ Found response in messages")
                break
            elif isinstance(content, list) and content:
                for item in content:
                    if isinstance(item, dict):
                        text = item.get('text', '')
                        if text:
                            response_text = text
                            print("   ✅ Found response in messages (list format)")
                            break
                    elif isinstance(item, str) and item.strip():
                        response_text = item
                        print("   ✅ Found response in messages (list format)")
                        break
                if response_text:
                    break
    
    # If not found in messages, check steps
    if not response_text:
        print("   ⚠️  Not found in messages, checking steps...")
        for step in reversed(steps):
            # Try inference step
            if step.get('type') == 'inference' or step.get('step_type') == 'inference':
                model_response = step.get('model_response', {})
                content = model_response.get('content', '')
                if isinstance(content, str) and content.strip():
                    response_text = content
                    print("   ✅ Found response in inference step")
                    break
            
            # Try any step with model_response
            if 'model_response' in step:
                model_response = step.get('model_response', {})
                content = model_response.get('content', '')
                if isinstance(content, str) and content.strip():
                    response_text = content
                    print("   ✅ Found response in step model_response")
                    break
    
    # Display the response
    print("\n" + "=" * 60)
    print("Agent Response")
    print("=" * 60)
    
    if response_text:
        print(f"\n💬 {response_text}\n")
        
        # Show tools used
        tools_used = []
        for step in steps:
            if step.get('type') == 'tool_call' or step.get('step_type') == 'tool_call':
                tool_call = step.get('tool_call', {})
                tool_name = tool_call.get('name', 'unknown')
                if tool_name not in tools_used:
                    tools_used.append(tool_name)
        
        if tools_used:
            print(f"🔧 Tools used: {', '.join(tools_used)}\n")
        else:
            print("💡 No tools were used in this response\n")
    else:
        print("\n⚠️  Could not extract response text")
        print("   💡 The turn may still be processing, or response format is unexpected")
        print("   💡 Check the steps above for more details")


In [ ]:
# Test with multiple queries
import json

def extract_tools_from_steps(steps):
    """Extract tool names from steps (checking both tool_execution and inference steps)"""
    tools_used = []
    for step in steps:
        step_type = step.get('type', '') or step.get('step_type', '')
        
        # Check tool_execution steps
        if step_type == 'tool_execution':
            tool_execution = step.get('tool_execution', {})
            tool_name = tool_execution.get('tool_name', '')
            if tool_name and tool_name not in tools_used:
                tools_used.append(tool_name)
        
        # Check inference steps for tool_calls
        elif step_type == 'inference':
            model_response = step.get('model_response', {})
            tool_calls = model_response.get('tool_calls', [])
            for tool_call in tool_calls:
                tool_name = tool_call.get('tool_name', '')
                if tool_name and tool_name not in tools_used:
                    tools_used.append(tool_name)
    
    return tools_used

def extract_response_text(data):
    """Extract response text from messages or steps"""
    messages = data.get('messages', [])
    steps = data.get('steps', [])
    response_text = None
    
    # Try messages first
    for msg in reversed(messages):
        if msg.get('role') == 'assistant':
            content = msg.get('content', '')
            if isinstance(content, str) and content.strip():
                response_text = content
                break
            elif isinstance(content, list) and content:
                for item in content:
                    if isinstance(item, dict):
                        text = item.get('text', '')
                        if text:
                            response_text = text
                            break
                    elif isinstance(item, str) and item.strip():
                        response_text = item
                        break
                if response_text:
                    break
    
    # If not found in messages, check steps
    if not response_text:
        for step in reversed(steps):
            if step.get('type') == 'inference' or step.get('step_type') == 'inference':
                model_response = step.get('model_response', {})
                content = model_response.get('content', '')
                if isinstance(content, str) and content.strip():
                    response_text = content
                    break
    
    return response_text

def retrieve_turn_with_retry(llamastack_url, agent_id, session_id, turn_id, max_retries=5, initial_wait=2):
    """Retrieve turn with retry logic"""
    for attempt in range(max_retries):
        wait_time = initial_wait * (2 ** attempt)  # Exponential backoff
        if attempt > 0:
            print(f"   ⏳ Retry {attempt}/{max_retries-1} (waiting {wait_time}s)...")
            time.sleep(wait_time)
        else:
            time.sleep(wait_time)
        
        try:
            response = httpx.get(
                f"{llamastack_url}/v1alpha/agents/{agent_id}/session/{session_id}/turn/{turn_id}",
                verify=False,
                timeout=30
            )
            response.raise_for_status()
            
            if response.text and response.text.strip() != 'null':
                data = response.json()
                steps = data.get('steps', [])
                
                # Check if turn is complete (has final inference step with content)
                is_complete = False
                for step in reversed(steps):
                    if step.get('type') == 'inference' or step.get('step_type') == 'inference':
                        model_response = step.get('model_response', {})
                        content = model_response.get('content', '')
                        if content and content.strip():
                            is_complete = True
                            break
                
                if is_complete or len(steps) > 0:
                    return data
                elif attempt < max_retries - 1:
                    continue  # Try again
            
        except Exception as e:
            if attempt < max_retries - 1:
                continue
            else:
                raise
    
    return None

print("=" * 60)
print("Testing Multiple Queries")
print("=" * 60)

test_queries = [
    "What collections are in the mcp_demo database?",
    "How many documents are in the incidents collection?",
    "Show me the first document from the incidents collection",
]

for i, query in enumerate(test_queries, 1):
    print(f"\n{'='*60}")
    print(f"Test {i}: {query}")
    print('='*60)
    
    try:
        # Create turn
        print("   📤 Creating turn...")
        turn_stream = client.alpha.agents.turn.create(
            agent_id=agent.agent_id,
            session_id=session.session_id,
            messages=[{"role": "user", "content": query}],
            stream=True
        )
        
        # Get turn_id
        print("   🔍 Extracting turn_id...")
        turn_id = None
        for chunk in turn_stream:
            if hasattr(chunk, 'event') and chunk.event and hasattr(chunk.event, 'payload'):
                payload = chunk.event.payload
                d = to_dict(payload)
                turn_id = d.get('turn_id')
                if not turn_id and 'step_details' in d:
                    step_details = to_dict(d['step_details'])
                    turn_id = step_details.get('turn_id')
                if turn_id:
                    break
        
        if turn_id:
            print(f"   ✅ Turn ID: {turn_id}")
            print("   ⏳ Retrieving turn results...")
            
            # Retrieve with retry
            data = retrieve_turn_with_retry(
                llamastack_url, 
                agent.agent_id, 
                session.session_id, 
                turn_id
            )
            
            if data:
                steps = data.get('steps', [])
                messages = data.get('messages', [])
                
                # Extract tools used
                tools_used = extract_tools_from_steps(steps)
                
                # Extract response
                response_text = extract_response_text(data)
                
                # Display results
                print(f"\n   📊 Results:")
                print(f"      Steps: {len(steps)}")
                print(f"      Messages: {len(messages)}")
                
                if tools_used:
                    print(f"      🔧 Tools used: {', '.join(tools_used)}")
                else:
                    print(f"      🔧 Tools used: None")
                    # Show step types for debugging
                    step_types = [step.get('type', '') or step.get('step_type', '') for step in steps]
                    if step_types:
                        print(f"      📋 Step types: {', '.join(step_types)}")
                
                if response_text:
                    # Truncate long responses
                    display_text = response_text[:300] + "..." if len(response_text) > 300 else response_text
                    print(f"      💬 Response: {display_text}")
                else:
                    print(f"      ⚠️  No response text found")
                    # Try to show what we have
                    if steps:
                        print(f"      💡 Check Part 5 and Part 6 for detailed step information")
            else:
                print(f"   ⚠️  Could not retrieve turn data after retries")
        else:
            print("   ⚠️  Could not get turn_id from stream")
            
    except Exception as e:
        print(f"   ❌ Error: {e}")
        import traceback
        traceback.print_exc()

print("\n✅ Multiple query test complete!")
print("\n💡 Tip: For detailed tool usage and step information, check Part 5 and Part 6 above.")
